<a href="https://colab.research.google.com/github/bearbearyu1223/llm-fine-tuning-playground/blob/main/finetune_falcon_7b_mental_health_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune [Falcon-7b-sharded model](https://huggingface.co/vilsonrodrigues/falcon-7b-sharded)** on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [1]:
#all installs
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install transformers==4.34.0
!pip install huggingface_hub==0.18.0
!pip install -q datasets bitsandbytes einops wandb


#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-h

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


##Notebook connection to Hugging face

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
notebook_login()

##Loading the dataset from hugging face and Formatting the Training Dataset

---



In [4]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
dataset = load_dataset(dataset_name)
dataset = dataset['train']
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 172
})

##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b model.


In [5]:
model_name = "vilsonrodrigues/falcon-7b-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

##Loading the tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

##Setting up the LoRA config

In [7]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

##Load the trainer

In [8]:
output_dir = "falcon_7b_LoRA_mental_health_model_Dec_7_23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to="wandb"
)

##Passing arguments to the SFTT trainer

In [9]:
max_seq_length = 512

instruction_template = "<HUMAN>:"
response_template = "<ASSISTANT>:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    dataset_text_field="text",
    data_collator=collator,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [10]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [14]:
import os
import time
import wandb
os.environ["WANDB_SILENT"] = "true"
start = time.time()
wandb.init(project="falcon-7b-peft-lora")

<IPython.core.display.Javascript object>

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


In [15]:
peft_model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.520400
20,1.205900
30,0.798000
40,0.407900
50,0.160900
60,0.048000
70,0.017000
80,0.009100
90,0.007100
100,0.006800


TrainOutput(global_step=100, training_loss=0.4181198463588953, metrics={'train_runtime': 811.9095, 'train_samples_per_second': 7.883, 'train_steps_per_second': 0.123, 'total_flos': 7.021990505930342e+16, 'train_loss': 0.4181198463588953, 'epoch': 36.36})

In [16]:
end=time.time()

In [17]:
time_taken=end-start
print(time_taken)

1002.4743490219116


##Save the model

In [ ]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [18]:
trainer.push_to_hub()

'https://huggingface.co/bearbearyu1223/falcon_7b_LoRA_mental_health_model_Dec_7_23/tree/main/'

##Inference

In [19]:
# Loading PEFT model
PEFT_MODEL = "bearbearyu1223/"+ output_dir
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [20]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """
  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to(torch.device("cuda:0"))
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=512, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_sharded:\n{peft_text_output}')


In [21]:
get_response("Are there cures for mental health problems?")

Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  **1.** While some mental health conditions can be cured (i.e., treated successfully with medication or therapy), others may not be cured but can be managed and controlled. People of all ages experience challenges with their mental health, and it's essential to seek help when needed.

Many factors contribute to mental health issues, such as genetics, life experiences, environmental factors, and more. The good news is that people often find significant relief through various forms of treatment and support. Mental health professionals collaborate with patients to develop personalized care plans that address specific concerns and improve well-being over time.

It's important

In [24]:
get_response("Are neurofeedback and biofeedback the same thing?")

Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are neurofeedback and biofeedback the same thing?

  ###Response:

   While neurofeedback and biofeedback may seem similar, they have distinct purposes and approaches. Neurofeedback is a form of brain training that aims to improve specific brainwave patterns. It involves taking ongoing tests of brain activity and providing feedback to patients about their brainwaves. Biofeedback, on the other hand, uses specialized equipment to measure certain bodily functions and then teaches techniques such as deep breathing to modulate them. These therapies can be used together or individually depending on the patient's needs.

 If you or someone you know is struggling with mental health or addiction, it's crucial to seek help from a qualified me

In [25]:
get_response(" What is insomnia disorder?")

Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question:  What is insomnia disorder?

  ###Response:

  **1.** Insomnia Disorder is when someone has difficulty falling asleep or staying asleep. They may also wake up early and have trouble going back to sleep. This must occur at least three nights per week for at least three months. People with Insomnia Disorder can feel tired even when they get enough sleep. 
  
**2.** Someone can be diagnosed with Insomnia Disorder if their symptoms interfere with daily life. Not sleeping well might make a person grumpy and unable to concentrate at work or school. It could affect how much restful sleep the person gets over time. 

Healthcare professionals can diagnose Insomnia Disorder by assessing a person's symptoms and their impact on their life. They